# Case Ifood

## Estrutura dos dados

### Biblioteca Básica

In [2]:
import pandas as pd
import numpy as np
import csv
from datetime import date
import io
import requests
import sqlite3
import sqlalchemy 
from sqlalchemy import create_engine

### Criação do database e tabelas

In [3]:
# Declarando a criação do database em um banco de dados sql
engine = sqlalchemy.create_engine('sqlite:///dw_ifood_analytics.db')

In [4]:
# Conectando a carga de dados que irá alimentar a tabela matriz

url = 'https://raw.githubusercontent.com/ifood/ifood-data-business-analyst-test/master/ml_project1_data.csv'
cont= requests.get(url).content
df = pd.read_csv(io.StringIO(cont.decode('utf-8')))

pd.set_option('max_columns', None)
df.head(1)

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
0,5524,1957,Graduation,Single,58138.0,0,0,2012-09-04,58,635,88,546,172,88,88,3,8,10,4,7,0,0,0,0,0,0,3,11,1


In [5]:
#df.to_sql('tb_ifood_case',engine) -- Etapa que transaforma a url recebida em tabela

## Análise Exploratória

### Conhecendo o público alvo
Esta etapa consiste em conhecer a base e o perfil que ela compõem; neste primeiro momento temos análises mais genéricas, com o intuito apenas de reconhecer os padrões básicos e algumas peculiaridades através de análises sóciodemograficas.

Apesar de simples, esta análise proporciona uma abordagem mais assertiva na construção das hipóteses e relações dos dados.

In [6]:
# Qual o grau de instrução da Base?

education = """
select
 Education,
 count(id) as User,
 (count(id) *100 / (select count(*) from tb_ifood_case)) as Perc
 
from tb_ifood_case

Group By
 Education
Order By
 User desc
"""
df1 = pd.read_sql(education,engine)

df1

,Education,User,Perc
0,Graduation,1127,50
1,PhD,486,21
2,Master,370,16
3,2n Cycle,203,9
4,Basic,54,2


In [7]:
# O estado civil da base é composto por quem?

marital = """
select
 Marital_Status,
 count(id) as User,
 (count(id) *100 / (select count(*) from tb_ifood_case)) as Perc
 
from tb_ifood_case

Group By
 Marital_Status
Order By
 User desc
"""
df2 = pd.read_sql(marital,engine)

df2

,Marital_Status,User,Perc
0,Married,864,38
1,Together,580,25
2,Single,480,21
3,Divorced,232,10
4,Widow,77,3
5,Alone,3,0
6,YOLO,2,0
7,Absurd,2,0


In [8]:
# Qual a idade da nossa Base?

Age = """
select
 case
  when (strftime('%Y', CURRENT_TIMESTAMP) - Year_Birth) between 0 and 20 then '0 a 20'
  when (strftime('%Y', CURRENT_TIMESTAMP) - Year_Birth) between 21 and 40 then '21 a 40'
  when (strftime('%Y', CURRENT_TIMESTAMP) - Year_Birth) between 41 and 60 then '41 a 60'
  when (strftime('%Y', CURRENT_TIMESTAMP) - Year_Birth) between 61 and 80 then '61 a 80'
  when (strftime('%Y', CURRENT_TIMESTAMP) - Year_Birth) > 80 then '81+'
 end as Gr_Age, 
 count(ID) as User,
 (count(ID) *100 / (select count(*) from tb_ifood_case)) as Perc
from tb_ifood_case

Group By
Gr_Age
Order BY
User desc
"""
df3 = pd.read_sql(Age,engine)

df3

,Gr_Age,User,Perc
0,41 a 60,1236,55
1,61 a 80,615,27
2,21 a 40,385,17
3,81+,4,0


In [9]:
# Qual a renda anual da nossa Base?

Income = """
select
 case
  when Income between 0 and 20000 then '0 a 20k'
  when Income between 21000 and 40000 then '21k a 40k'
  when Income between 41000 and 60000 then '41k a 60k'
  when Income between 61000 and 80000 then '61k a 80k'
  when Income > 80 then '81+'
 end as Gr_Income, 
 count(ID) as User,
 (count(ID) *100 / (select count(*) from tb_ifood_case)) as Perc
from tb_ifood_case

Group By
Gr_Income
Order BY
User desc
"""
df4 = pd.read_sql(Income,engine)

df4

,Gr_Income,User,Perc
0,41k a 60k,612,27
1,61k a 80k,596,26
2,21k a 40k,589,26
3,81+,292,13
4,0 a 20k,127,5
5,None,24,1


In [10]:
# Quantos usuários tem criança em casa

kidhome = """
select 
 count(ID) as User,
 (count(ID) *100 / (select count(*) from tb_ifood_case)) as Perc
from tb_ifood_case

where
 Kidhome > 0

Order BY
User desc
"""
df5 = pd.read_sql(kidhome,engine)

df5

,User,Perc
0,947,42


In [11]:
# Quantos usuários tem adolescente em casa?

teenhome = """
select 
 count(ID) as User,
 (count(ID) *100 / (select count(*) from tb_ifood_case)) as Perc
from tb_ifood_case

where
 teenhome > 0

Order BY
User desc
"""
df5 = pd.read_sql(teenhome,engine)

df5

,User,Perc
0,1082,48


In [12]:
# Quantos usuários tem adolescente ou crianças em casa?

family = """
select 
 count(ID) as User,
 (count(ID) *100 / (select count(*) from tb_ifood_case)) as Perc
from tb_ifood_case

where
 teenhome > 0
 or kidhome >0

Order BY
User desc
"""
df6 = pd.read_sql(family,engine)

df6

,User,Perc
0,1602,71


#### Conclusões

Podemos concluir que esta base tem um perfil mais maduro, com uma faixa predominante de 41 a 60 anos (55%), com faixas de renda anuais estáveis, apessar da pouca diferença dentro da segmentação. É um público mais familiar, 63% da base é casado (a) ou tem união estável e 71% tem criança ou adolescente em casa, o que pode nos dar bons indicios de padrão no comportamento de consumo.

### Conhecendo os perfis de compra - Wines
Nessa fase começaremos a relacionar os perfis sociodemograficos a comportamentos de compra ou as categorias. como o desdobramento pode levar inumeras células de hipóteses e análises, focaremos na exploratória dentro da categoria de vinhos, que tem a maior representatividade (montante) dentro das categorias. 

In [13]:
# Falando em vinhos, qual o spending médio por usuário?

spd_wine = """
select 
 count(ID) as User,
 sum(MntWines) as amount,
 sum(MntWines)/count(ID) as spending
from tb_ifood_case
where
MntWines > 0
"""
df7 = pd.read_sql(spd_wine,engine)

df7

,User,amount,spending
0,2227,680816,305


In [14]:
# Publicos mais velhos tem um spending maior em vinhos do que outras faixas?

age_wine = """
select 
 case
  when (strftime('%Y', CURRENT_TIMESTAMP) - Year_Birth) between 0 and 20 then '0 a 20'
  when (strftime('%Y', CURRENT_TIMESTAMP) - Year_Birth) between 21 and 40 then '21 a 40'
  when (strftime('%Y', CURRENT_TIMESTAMP) - Year_Birth) between 41 and 60 then '41 a 60'
  when (strftime('%Y', CURRENT_TIMESTAMP) - Year_Birth) between 61 and 80 then '61 a 80'
  when (strftime('%Y', CURRENT_TIMESTAMP) - Year_Birth) > 80 then '81+'
 end as Gr_Age,
 count(ID) as User,
 sum(MntWines) as amount,
 sum(MntWines)/count(ID) as spending,
 (sum(MntWines) *100 / (select sum(MntWines) from tb_ifood_case)) as perc
from tb_ifood_case
where
MntWines > 0

Group by
Gr_Age
Order By
perc desc
"""
df8 = pd.read_sql(age_wine,engine)

df8

,Gr_Age,User,amount,spending,perc
0,41 a 60,1230,352533,286,51
1,61 a 80,613,229880,375,33
2,21 a 40,380,97481,256,14
3,81+,4,922,230,0


In [15]:
# Publicos com maior grau de instrução tendem a gastar mais em vinhos?

edu_wine = """
select 
 Education,
 count(ID) as User,
 sum(MntWines) as amount,
 sum(MntWines)/count(ID) as spending,
 (sum(MntWines) *100 / (select sum(MntWines) from tb_ifood_case)) as perc
from tb_ifood_case
where
MntWines > 0

Group by
Education
Order By
perc desc
"""
df9 = pd.read_sql(edu_wine,engine)

df9

,Education,User,amount,spending,perc
0,Graduation,1124,320371,285,47
1,PhD,486,196585,404,28
2,Master,370,123238,333,18
3,2n Cycle,199,40231,202,5
4,Basic,48,391,8,0


In [16]:
# Quanto maior a renda, maior o spending em vinhos?

Inc_wine = """
select 
 case
  when Income between 0 and 20000 then '0 a 20k'
  when Income between 21000 and 40000 then '21k a 40k'
  when Income between 41000 and 60000 then '41k a 60k'
  when Income between 61000 and 80000 then '61k a 80k'
  when Income > 80 then '81+'
 end as Gr_Income,
 count(ID) as User,
 sum(MntWines) as amount,
 sum(MntWines)/count(ID) as spending,
 (sum(MntWines) *100 / (select sum(MntWines) from tb_ifood_case)) as perc
from tb_ifood_case
where
MntWines > 0

Group by
Gr_Income
Order By
perc desc
"""
df10 = pd.read_sql(Inc_wine,engine)

df10

,Gr_Income,User,amount,spending,perc
0,61k a 80k,596,339556,569,49
1,81+,289,163088,564,23
2,41k a 60k,612,151656,247,22
3,21k a 40k,585,20563,35,3
4,0 a 20k,121,1220,10,0
5,None,24,4733,197,0


In [17]:
# o Spending de quem teve contato com alguma campanha é maior?

camp_wine = """
select 
 'Camp' as tipo,
 count(ID) as User,
 sum(MntWines) as amount,
 sum(MntWines)/count(ID) as spending,
 (sum(MntWines) *100 / (select sum(MntWines) from tb_ifood_case)) as perc
from tb_ifood_case
where
MntWines > 0
and AcceptedCmp1 = 1
or AcceptedCmp2 = 1
or AcceptedCmp3 = 1
or AcceptedCmp4 = 1
or AcceptedCmp5 = 1

union

select
 'Sem_Camp' as tipo,
 count(ID) as User,
 sum(MntWines) as amount,
 sum(MntWines)/count(ID) as spending,
 (sum(MntWines) *100 / (select sum(MntWines) from tb_ifood_case)) as perc
from tb_ifood_case
where
MntWines > 0
and AcceptedCmp1 = 0
and AcceptedCmp2 = 0
and AcceptedCmp3 = 0
and AcceptedCmp4 = 0
and AcceptedCmp5 = 0

Group By
tipo
Order By
perc desc

"""
df11 = pd.read_sql(camp_wine,engine)

df11

,tipo,User,amount,spending,perc
0,Sem_Camp,1765,397792,225,58
1,Camp,463,283024,611,41


#### Conclusões

Vinhos não só tem um montante de vendas alto como um spending maior (gasto médio por usuário), sua base é composta por um publico mais velho (41 a 60 anos), com uma representatividade de 50% do montante de vendas; um ponto interessante é o spending do publico acima de 60 anos, $89 a mais que o publico predominante.

Constatamos também que a instrução tem um peso nessa categoria, PHD's e Mestres tendem a gastar mais do que graduados, um delta de $150 a mais, e se analisarmos o montante desse grupo, se iguala ao dos graduados, que tem a maior representatividade de base.

Pra fechar esse tópico, entrevistados que tiveram contato com alguma campanha tem um spending maior do que os que não tiveram, um delta de $ 386, ou seja, bases que mantem o contato com a marca por meio de campanhas tendem a gastar mais no produto

### Conhecendo os canais de vendas

Neste tópico analisaremos brevemente os canais, fazendo uma análise exploratória de suas representatividades e a distribuição dos purchases

In [18]:
# Qual a média de compras com desconto por usuário?

deal = """
select 
 count(ID) as User,
 sum(NumDealsPurchases) as Purchase,
 sum(NumDealsPurchases)/count(ID) as Purchase_User
from tb_ifood_case

where
NumDealsPurchases > 0

"""
df12 = pd.read_sql(deal,engine)

df12

,User,Purchase,Purchase_User
0,2194,5208,2


In [19]:
# Qual a média de compras na web por usuário?

web = """
select 
 count(ID) as User,
 sum(NumWebPurchases) as Purchase,
 sum(NumWebPurchases)/count(ID) as Purchase_User
from tb_ifood_case

where
NumWebPurchases > 0

"""
df13 = pd.read_sql(web,engine)

df13

,User,Purchase,Purchase_User
0,2191,9150,4


In [20]:
# Qual a média de compras no catalogo por usuário?

catalog = """
select 
 count(ID) as User,
 sum(NumCatalogPurchases) as Purchase,
 sum(NumCatalogPurchases)/count(ID) as Purchase_User
from tb_ifood_case

where
NumCatalogPurchases > 0

"""
df14 = pd.read_sql(catalog,engine)

df14

,User,Purchase,Purchase_User
0,1654,5963,3


In [21]:
# Qual a média de compras na loja por usuário?

store = """
select 
 count(ID) as User,
 sum(NumStorePurchases) as Purchase,
 sum(NumStorePurchases)/count(ID) as Purchase_User
from tb_ifood_case

where
NumStorePurchases > 0

"""
df15 = pd.read_sql(store,engine)

df15

,User,Purchase,Purchase_User
0,2225,12970,5


In [22]:
# Qual a média de acessos por usuário?

web_ac = """
select 
 count(ID) as User,
 sum(NumWebVisitsMonth) as Visits,
 sum(NumWebVisitsMonth)/count(ID) as Visits_User
from tb_ifood_case

where
NumWebVisitsMonth > 0

"""
df16 = pd.read_sql(web_ac,engine)

df16

,User,Visits,Visits_User
0,2229,11909,5


In [23]:
# Qual o percentual de compra na web de quem visita?

web_purchase = """
select 
 count(ID) as User,
 sum(NumWebVisitsMonth) as Visits,
 sum(NumWebPurchases) as Purchase,
 (sum(NumWebPurchases)*100/sum(NumWebVisitsMonth)) as Conv
from tb_ifood_case

where
NumWebVisitsMonth > 0

"""
df17 = pd.read_sql(web_purchase,engine)

df17


,User,Visits,Purchase,Conv
0,2229,11909,9116,76


#### Conclusões

Store é o canal com maior numeros de compras, em média são 5 compras por usuário nesse canal. A web vem em segundo lugar com  9150 compras e uma média de 4 compras por usuário.

Usuários que visitam a pagina tendem a comprar na web, em percentuais 76% das visitas converteram em compra.